## Computing PCA using RDDs

##  PCA

The vectors that we want to analyze have length, or dimension, of 365, corresponding to the number of 
days in a year.

We will perform [Principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis)
on these vectors. There are two steps to this process:

1. Computing the covariance matrix: this is a  simple computation. However, it takes a long time to compute and it benefits from using an RDD because it involves all of the input vectors.
2. Computing the eigenvector decomposition. this is a more complex computation, but it takes a fraction of a second because the size to the covariance matrix is $365 \times 365$, which is quite small. We do it on the head node usin `linalg`

### Computing the covariance matrix
Suppose that the data vectors are the column vectors denoted $x$ then the covariance matrix is defined to be
$$
E(x x^T)-E(x)E(x)^T
$$

Where $x x^T$ is the **outer product** of $x$ with itself.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

### The effect of  `nan`s in arithmetic operations
* We use an RDD of numpy arrays, instead of Dataframes.
* Why? Because numpy treats `nan` entries correctly:
  * In numpy `5+nan=5` while in dataframes `5+nan=nan`

### Performing Cov matrix on vectors with NaNs
As it happens, we often get vectors $x$ in which some, but not all, of the entries are `nan`. 
Suppose that we want to compute the mean of the elements of $x$. If we use `np.mean` we will get the result `nan`. A useful alternative is to use `np.nanmean` which removes the `nan` elements and takes the mean of the rest.

import numpy as np
X=np.array([1,1,1,2])
print 'mean of',X,'=',np.mean(X)
print 'nanmean of',X,'=',np.nanmean(X)
X=np.array([1,1,np.NaN,2])
print 'mean of',X,'=',np.mean(X)
print 'nanmean of',X,'=',np.nanmean(X)

#### When should you not use `np.nanmean` ?
Using `n.nanmean` is equivalent to assuming that choice of which elements to remove is independent of the values of the elements. 
* Example of bad case: suppose the larger elements have a higher probability of being `nan`. In that case `np.nanmean` will under-estimate the mean

#### Computing the covariance  when there are `nan`s
The covariance is a mean of outer products.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

x1=np.array([1,np.NaN,3,4,5])
x2=np.array([2,3,4,np.NaN,6])
stacked=np.array([np.outer(x1,x1),np.outer(x2,x2)])
stacked

np.nanmean(stacked,axis=0)

In [3]:
import findspark
findspark.init()
from pyspark import SparkContext

#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark.sql import *
sqlContext = SQLContext(sc)

In [4]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

### Climate data

The data we will use here comes from [NOAA](https://www.ncdc.noaa.gov/). Specifically, it was downloaded from This [FTP site](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/).

There is a large variety of measurements from all over the world, from 1870 will 2012.
in the directory `../../Data/Weather` you will find the following useful files:

* data-source.txt: the source of the data
* ghcnd-readme.txt: A description of the content and format of the data
* ghcnd-stations.txt: A table describing the Meteorological stations.



### Data cleaning

* Most measurements exists only for a tiny fraction of the stations and years. We therefor restrict our use to the following measurements:
```python
['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']
```

* 8 We consider only measurement-years that have at most 50 `NaN` entries

* We consider only measurements in the continential USA

* We partition the stations into 256 geographical rectangles, indexed from BBBBBBBB to SSSSSSSS. And each containing about 12,000 station,year pairs.

In [9]:
#file_index='BBBSBBBB'
file_index='SBBBSBSS'
data_dir='../../Data/Weather'

filebase='US_Weather_%s'%file_index
!rm -rf $data_dir/$filebase*

c_filename=filebase+'.csv.gz'
u_filename=filebase+'.csv'

command="curl https://mas-dse-open.s3.amazonaws.com/Weather/small/%s > %s/%s"%(c_filename,data_dir,c_filename)
print command
!$command
!ls -lh $data_dir/$c_filename

curl https://mas-dse-open.s3.amazonaws.com/Weather/small/US_Weather_SBBBSBSS.csv.gz > ../../Data/Weather/US_Weather_SBBBSBSS.csv.gz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3635k  100 3635k    0     0   251k      0  0:00:14  0:00:14 --:--:--  186k
-rw-r--r--  1 collinli  staff   3.5M May 14 21:36 ../../Data/Weather/US_Weather_SBBBSBSS.csv.gz


In [10]:
#unzip
!gunzip -c $data_dir/$c_filename > $data_dir/$u_filename
import pickle
List=pickle.load(open(data_dir+'/'+u_filename,'rb'))
len(List)

12351

In [11]:
df=sqlContext.createDataFrame(List)
print df.count()
df.show(5)

12351
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|    885.4| 47.3389|-108.4983|       TMAX|USC00248498|     2|[80 54 90 55 00 5...|1908.0|SBBBSBSS|
|    885.4| 47.3389|-108.4983|       TMAX|USC00248498|    25|[00 CF F0 56 90 5...|1909.0|SBBBSBSS|
|    885.4| 47.3389|-108.4983|       TMAX|USC00248498|    29|[80 C9 00 7E 00 7...|1910.0|SBBBSBSS|
|    885.4| 47.3389|-108.4983|       TMAX|USC00248498|    37|[00 7E 00 7E 80 4...|1911.0|SBBBSBSS|
|    885.4| 47.3389|-108.4983|       TMAX|USC00248498|    40|[E0 D5 00 7E E0 D...|1912.0|SBBBSBSS|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
only showing top 5 rows



In [12]:
#store dataframe as parquet file
outfilename=data_dir+'/'+filebase+'.parquet'
!rm -rf $outfilename
df.write.save(outfilename)

In [13]:
# Compare file sizes
!du -sh $data_dir/$filebase*

 12M	../../Data/Weather/US_Weather_SBBBSBSS.csv
3.6M	../../Data/Weather/US_Weather_SBBBSBSS.csv.gz
4.5M	../../Data/Weather/US_Weather_SBBBSBSS.parquet


In [14]:
from time import time
t=time()

N=sc.defaultParallelism
print 'Number of executors=',N
print 'took',time()-t,'seconds'

Number of executors= 3
took 0.00356793403625 seconds


In [15]:
measurements=['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']
#measurements=['TMAX','TMIN']

In [16]:
sqlContext.registerDataFrameAsTable(df,'weather') #using older sqlContext instead of newer (V2.0) sparkSession

In [18]:
from numpy import linalg as LA
STAT={}  # dictionary storing the statistics for each measurement
Clean_Tables={}

for meas in measurements:
    t=time()
    Query="SELECT * FROM weather\n\tWHERE measurement = '%s'"%(meas)
    print Query
    df = sqlContext.sql(Query)
    data=df.rdd.map(lambda row: unpackArray(row['vector'],np.float16))
    #get very basic statistics
    STAT[meas]=computeOverAllDist(data)   # Compute the statistics 

    # compute covariance matrix
    
    OUT=computeCov(data)

    #find PCA decomposition
    eigval,eigvec=LA.eig(OUT['Cov'])

    # collect all of the statistics in STAT[meas]
    STAT[meas]['eigval']=eigval
    STAT[meas]['eigvec']=eigvec
    STAT[meas].update(OUT)

    print 'time for',meas,'is',time()-t

SELECT * FROM weather
	WHERE measurement = 'TMAX'
shape of E= (365,) shape of NE= (365,)
time for TMAX is 27.1941740513
SELECT * FROM weather
	WHERE measurement = 'SNOW'
shape of E= (365,) shape of NE= (365,)
time for SNOW is 22.7155828476
SELECT * FROM weather
	WHERE measurement = 'SNWD'
shape of E= (365,) shape of NE= (365,)
time for SNWD is 18.622756958
SELECT * FROM weather
	WHERE measurement = 'TMIN'
shape of E= (365,) shape of NE= (365,)
time for TMIN is 25.9577829838
SELECT * FROM weather
	WHERE measurement = 'PRCP'
shape of E= (365,) shape of NE= (365,)
time for PRCP is 30.1999418736
SELECT * FROM weather
	WHERE measurement = 'TOBS'
shape of E= (365,) shape of NE= (365,)
time for TOBS is 14.9634530544


In [114]:
Query1="SELECT latitude,vector FROM weather WHERE measurement = 'TMAX'"
Query2="SELECT DISTINCT station,year FROM weather WHERE measurement = 'TMAX'"
print Query1
df1 = sqlContext.sql(Query1)
df2 = sqlContext.sql(Query2)
data1=df1.rdd.map(lambda row: unpackArray(row['vector'],np.float16))
l2 = df2.rdd.collect()
print(len(l2))

Stat = {}
c = 0
for l in l2:
    Query3="SELECT * FROM weather\n\tWHERE measurement = 'TMAX' AND station = '%s' AND year='%f'"%(l[0],l[1])
    Query4="SELECT * FROM weather\n\tWHERE measurement = 'TMIN' AND station = '%s' AND year='%f'"%(l[0],l[1])
    df3 = sqlContext.sql(Query3)
    df4 = sqlContext.sql(Query4)
    if df3.count() != 0 and df4.count() != 0:
        c += 1
        print(c)


SELECT latitude,vector FROM weather WHERE measurement = 'TMAX'
2386
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260


In [109]:
data7=df_concat.rdd.filter(lambda r: r['measurement'] == "TDIFF")
STAT["TDIFF"]=computeOverAllDist(data7)
# compute covariance matrix
    
OUT=computeCov(data7)

#find PCA decomposition
eigval,eigvec=LA.eig(OUT['Cov'])

# collect all of the statistics in STAT[meas]
STAT["TDIFF"]['eigval']=eigval
STAT["TDIFF"]['eigvec']=eigvec
STAT["TDIFF"].update(OUT)

[Row(elevation=808.0, latitude=47.5667, longitude=-106.7, measurement=u'TMAX', station=u'USC00244007', undefs=28, vector=bytearray(b'\x00~\x00~\x00~\x00~\x80Q\x80T\xe0T\x80Q\x80\xd4\x80Q@\xcc\xe0\xd4\x00\xcf\x80I\xa0\xd3\x00~\x00~\x00~\x00~\x00~@L\x00\x000\xd4\x00~\x00~\xa0\xd6\xa0S\x80T\x00S\x80T\x00X(X\xe0U\x00O\x80I\x80\xcd\x00\xc6\x00\x00\x80Q\xe0P0U\x00X\xb8YXX@L\x80I\x80I\x80Q\x00\x00\x00~\x00~\x80Q\x80\xc9\x80I\x00S\x00S\xe0U\xa0W\xf0V\xa0V\xe0T\x00S@L\x80\xcd\x80\xd1\xe0\xd0\x00\xd3\x80\xc9\x00O@L\xa0S\x80I\x80I\x00\x00\x80I\x00F\x80M\x00O\x80Q\x80M\x80I\x00O\x00O\x80I0T\x00F\x00\xcf@\xd2\x00\xc6\xa0S\xa0V\xb0XPW\x80T\xa0WpZpZ0T0T\xa0S\x80Q\xb8Y0U\xa0W0T\xe0X\xa0V\xf0V0T\xe0U\x80X\x98Z\xe8Y(X\xe0T\xa0W\xa0S@V\xf0V\xb8YH[\xa0[\xc8\\\xa0[\x90Y\xe0P\x00X\xa0V(X\xe8Y [\xb0X\xe0X@Z\x84\\x[8Y\xe0U\x90U\xe0X\x98Zx[\x00X\xb8Y8Y(X\x10Z@Z\xb8Y\xe8Y\xf0ZpZ\xa0[x[\xe8Y@Z\x98ZPW\xb0XXX\xf0Z [ [\xd0[ [ [X\\\x14\\`Y\xe8Y\xb8Y [\xb8Y\x00~\xe8Y [\x00~H[x[\xf0Z [\x00\\l\\\xb0\\\xdc\\\x84\\\x08]\

In [19]:
from pickle import dump
filename=data_dir+'/STAT_%s.pickle'%file_index
dump((STAT,STAT_Descriptions),open(filename,'wb'))
